In [ ]:
import argparse
import random
import torch
import pandas as pd
import numpy as np
import sys
import io
from contextlib import redirect_stdout
from torch import nn
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torch.optim import Adam
from tqdm import tqdm

# Set seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)


# Helper functions to load and process the data into desired format needed for MF
# For MF we need a "model ID" either in the form of name or index and so we use the tabular data instead of tensors
def load_and_process_data(train_data, test_data, batch_size=64 , model_use_l = 0,model_use_r = 112):
    # NOTE: Due to the nature of the embedding layer we need to take max prompt ID from both train and test data
    # But during training we won't be using test question
    num_prompts = int(max(max(train_data["prompt_id"]), max(test_data["prompt_id"]))) + 1

    model_ids = test_data["model_id"].unique()
    selected_model_ids = model_ids[model_use_l:model_use_r]

    train_data = train_data[train_data["model_id"].isin(selected_model_ids)]
    test_data = test_data[test_data["model_id"].isin(selected_model_ids)]

    class CustomDataset(Dataset):
        def __init__(self, data):
            model_ids = pd.factorize(data["model_id"])[0]  # turn model_ids into int
            model_ids = torch.tensor(model_ids, dtype=torch.int64)
            
            unique_ids, inverse_indices = torch.unique(model_ids, sorted=True, return_inverse=True)
            id_to_rank = {id.item(): rank for rank, id in enumerate(unique_ids)}
            ranked_model_ids = torch.tensor([id_to_rank[id.item()] for id in model_ids])
            self.models = ranked_model_ids

            prompts = pd.factorize(data["prompt_id"])[0]
            self.prompts = torch.tensor(prompts, dtype=torch.int64)
            labels = pd.factorize(data["label"])[0]
            self.labels = torch.tensor(labels, dtype=torch.int64)

            self.num_models = len(data["model_id"].unique())
            self.num_prompts = num_prompts
            self.num_classes = len(data["label"].unique())

        def get_num_models(self):
            return self.num_models

        def get_num_prompts(self):
            return self.num_prompts

        def get_num_classes(self):
            return self.num_classes

        def __len__(self):
            return len(self.models)

        def __getitem__(self, index):
            return self.models[index], self.prompts[index], self.labels[index]

        def get_dataloaders(self, batch_size):
            return DataLoader(self, batch_size, shuffle=False)

    train_dataset = CustomDataset(train_data)
    test_dataset = CustomDataset(test_data)

    train_loader = train_dataset.get_dataloaders(batch_size)
    test_loader = test_dataset.get_dataloaders(batch_size)

    return train_loader, test_loader

class TextMF(nn.Module):
    def __init__(self, question_embeddings, model_embedding_dim, alpha, num_models, num_prompts, text_dim=768, num_classes=2):
        super(TextMF, self).__init__()
        # Model embedding network
        self.P = nn.Embedding(num_models, model_embedding_dim)

        # Question embedding network
        self.Q = nn.Embedding(num_prompts, text_dim).requires_grad_(False)
        self.Q.weight.data.copy_(question_embeddings)
        self.text_proj = nn.Linear(text_dim, model_embedding_dim)

        # Noise/Regularization level
        self.alpha = alpha
        self.classifier = nn.Linear(model_embedding_dim, num_classes)

    def forward(self, model, prompt, test_mode=False):
        p = self.P(model)
        q = self.Q(prompt)
        if not test_mode:
            # Adding a small amount of noise in question embedding to reduce overfitting
            q += torch.randn_like(q) * self.alpha
        q = self.text_proj(q)
        return self.classifier(p * q)
    
    @torch.no_grad()
    def predict(self, model, prompt):
        logits = self.forward(model, prompt, test_mode=True) # During inference no noise is applied
        return torch.argmax(logits, dim=1)
    
def evaluate(net, test_loader, device):
    net.eval()
    loss_fn = nn.CrossEntropyLoss(reduction="sum")
    total_loss = 0
    correct = 0
    num_samples = 0

    with torch.no_grad():
        for models, prompts, labels in test_loader:
            models, prompts, labels = models.to(device), prompts.to(device), labels.to(device)
            logits = net(models, prompts)
            loss = loss_fn(logits, labels)
            pred_labels = net.predict(models, prompts)
            correct += (pred_labels == labels).sum().item()
            total_loss += loss.item()
            num_samples += labels.shape[0]

    mean_loss = total_loss / num_samples
    accuracy = correct / num_samples
    net.train()
    return mean_loss, accuracy

def test(net,test_loader,device) :

    test_loss, test_accuracy = evaluate(net, test_loader, device)
    print(f'Test Loss: {test_loss} , Test Accuracy: {test_accuracy}')

def load_model(net, path, device):
    net.load_state_dict(torch.load(path, map_location=device))
    print(f"Model loaded from {path}")


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--embedding_dim", type=int, default=232)
    parser.add_argument("--alpha", type=float, default=0.05)
    parser.add_argument("--batch_size", type=int, default=2048)
    parser.add_argument("--num_epochs", type=int, default=50)
    parser.add_argument("--learning_rate", type=float, default=1e-4)

    parser.add_argument("--train_data_path", type=str, default="data/train.csv")
    parser.add_argument("--test_data_path", type=str, default="data/test.csv")
    parser.add_argument("--question_embedding_path", type=str, default="data/question_embeddings.pth")

    parser.add_argument("--embedding_save_path", type=str, default="data/model_embeddings.pth")
    parser.add_argument("--model_save_path", type=str, default=None)
    parser.add_argument("--model_load_path", type=str, default="data/saved_model.pth")
    parser.add_argument("--loader",type = int,default=0)
    
    parser.add_argument("--output_save_path",type = str ,default  = None)

    args = parser.parse_args(["--loader","1","--output_save_path","output/result.txt","--model_load_path","data/model_with_fir90.pth"])

    captured_output = io.StringIO()
    with redirect_stdout(captured_output):
    
        print("Loading dataset...")
        train_data = pd.read_csv(args.train_data_path)
        test_data = pd.read_csv(args.test_data_path)
        question_embeddings = torch.load(args.question_embedding_path)
        num_prompts = question_embeddings.shape[0]
        num_models = len(test_data["model_id"].unique())
        model_names = list(np.unique(list(test_data["model_name"])))

        train_loader0, test_loader0 = load_and_process_data(train_data, test_data, batch_size=args.batch_size , model_use_l = 0,model_use_r = 90)
        train_loader1, test_loader1 = load_and_process_data(train_data, test_data, batch_size=args.batch_size , model_use_l = 90,model_use_r = 112)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        loader = train_loader0 
        if args.loader == 1:
            loader = train_loader1 
        elif args.loader == 2 :
            loader = test_loader0
        elif args.loader == 3 :
            loader = test_loader1
        print(f'loader : {args.loader}')

        print("Initializing model...")
        model = TextMF(question_embeddings=question_embeddings, 
                        model_embedding_dim=args.embedding_dim, alpha=args.alpha,
                        num_models=num_models, num_prompts=num_prompts)
        load_model(model,args.model_load_path,device)

        print("Testing models...")
        test(model,loader,device=device)

    with open(args.output_save_path, "a", encoding="utf-8") as f:
        f.write(captured_output.getvalue())
        

C:\Users\21296\AppData\Local\Temp\ipykernel_126360\2248661777.py:166: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  question_embeddings = torch.load(args.question_embedding_

: 